## 1. 简介
- ThreadLocal 也叫 "线程本地变量"、"线程局部变量"
  - 其作用域覆盖线程，而不是某个具体任务
  - 其"自然"的生命周期与线程的生命周期"相同"(但在 JDK 实现中比线程的生命周期更短，减少了内存泄漏的可能)

- ThreadLocal 的实现中最重要的一点就是: "将线程相关的域封闭在当前线程实例中"，虽然域仍然在任务对象中声明、但 set 和 get 方法却与任务对象无关。
- **ThreadLocal 适用于场景**
  - 每个线程需要有自己单独的实例
  - 实例需要在多个方法中共享，但不希望被多线程共享

---

## 2. 原理
### 2.1 方案1(未采纳)：ThreadLocal 维护 Thread 与实例的映射
- ThreadLocal 维护一个 Map，键是 Thread，值是它在该 Thread 内的实例。线程通过该 ThreadLocal 的 get() 方案获取实例时，只需要以线程为键，从 Map 中找出对应的实例即可。
- 存在的问题
  - 增加线程与减少线程均需要写 Map，故需保证该 Map 线程安全，需要加锁处理。(锁的问题，是 JDK 未采用该方案的一个原因)
  - 线程结束时，需要保证它所访问的所有 ThreadLocal 中对应的映射均删除，否则可能会引起内存泄漏。

![image](ThreadLocal1.png)


### 2.2 方案2：Thread 维护 ThreadLocal 与实例的映射
- 如果该 Map 由 Thread 维护，从而使得每个 Thread 只访问自己的 Map，那就不存在多线程写的问题，也就不需要锁。

![image](ThreadLocal2.png)

---

## 3. ThreadLocal 源码分析
### 3.1 构造函数
- ThreadLocal 是一个泛型类，保证可以接受任何类型的对象

```
    /**
     * Creates a thread local variable.
     * @see #withInitial(java.util.function.Supplier)
     */
    public ThreadLocal() {
    }
```

### 3.2 ThreadLocal 的 set 方法
- 根据当前线程 t 获取一个 ThreadLocalMap 类型的 map，真正的 value 保存在这个 map 中

```
/**
     * Sets the current thread's copy of this thread-local variable
     * to the specified value.  Most subclasses will have no need to
     * override this method, relying solely on the {@link #initialValue}
     * method to set the values of thread-locals.
     *
     * @param value the value to be stored in the current thread's copy of
     *        this thread-local.
     */
    public void set(T value) {
        Thread t = Thread.currentThread();
        ThreadLocalMap map = getMap(t);
        if (map != null)
            // 使用 this 引用作为 key，既做到了变量相关，又满足 key 不可变的要求。
            map.set(this, value);
        else
            createMap(t, value);
    }
```

### 3.3 ThreadLocal 的 getMap 方法
- 获取 Thread 实例中的 threadLocals

```
    /**
     * Get the map associated with a ThreadLocal. Overridden in
     * InheritableThreadLocal.
     *
     * @param  t the current thread
     * @return the map
     */
    ThreadLocalMap getMap(Thread t) {
        return t.threadLocals;
    }
```

---

### 3.4 ThreadLocal 的 get 方法
- 通过 getMap(Thread t) 方法获取 thread 对应的 ThreadLocalMap

```
    /**
     * Returns the value in the current thread's copy of this
     * thread-local variable.  If the variable has no value for the
     * current thread, it is first initialized to the value returned
     * by an invocation of the {@link #initialValue} method.
     *
     * @return the current thread's value of this thread-local
     */
    public T get() {
        Thread t = Thread.currentThread();
        ThreadLocalMap map = getMap(t);
        if (map != null) {
            ThreadLocalMap.Entry e = map.getEntry(this);
            if (e != null) {
                @SuppressWarnings("unchecked")
                T result = (T)e.value;
                return result;
            }
        }
        return setInitialValue();
    }
```

### 3.5 ThreadLocal 的 createMap 方法
- 初始化

```
    /**
     * Create the map associated with a ThreadLocal. Overridden in
     * InheritableThreadLocal.
     *
     * @param t the current thread
     * @param firstValue value for the initial entry of the map
     */
    void createMap(Thread t, T firstValue) {
        t.threadLocals = new ThreadLocalMap(this, firstValue);
    }
```

---

## 4. ThreadLocalMap 源码
### 4.1 内部类
- ThreadLocalMap 没有实现 Map 接口，但具有常见 Map 实现类的大部分属性（与 HashMap 不同，hash 重复时在 table 里顺序遍历）
- Entry 继承了一个 ThreadLocal 泛型的 WeakReference 引用
  - 保存的是 ThreadLocal 变量
  - WeakReference 是为了方便垃圾回收

```
/**
 * ThreadLocalMap is a customized hash map suitable only for
 * maintaining thread local values. No operations are exported
 * outside of the ThreadLocal class. The class is package private to
 * allow declaration of fields in class Thread.  To help deal with
 * very large and long-lived usages, the hash table entries use
 * WeakReferences for keys. However, since reference queues are not
 * used, stale entries are guaranteed to be removed only when
 * the table starts running out of space.
 */
static class ThreadLocalMap {

    /**
     * The entries in this hash map extend WeakReference, using
     * its main ref field as the key (which is always a
     * ThreadLocal object).  Note that null keys (i.e. entry.get()
     * == null) mean that the key is no longer referenced, so the
     * entry can be expunged from table.  Such entries are referred to
     * as "stale entries" in the code that follows.
     */
    static class Entry extends WeakReference<ThreadLocal<?>> {
        /** The value associated with this ThreadLocal. */
        Object value;

        Entry(ThreadLocal<?> k, Object v) {
            super(k);
            value = v;
        }
    }

    /**
     * The initial capacity -- MUST be a power of two.
     */
    private static final int INITIAL_CAPACITY = 16;

    /**
     * The table, resized as necessary.
     * table.length MUST always be a power of two.
     */
    private Entry[] table;

    /**
     * The number of entries in the table.
     */
    private int size = 0;

    // 省略其他代码
    ......
}
```

### 4.2 getEntry 方法
- 获取该 ThreadLocal 在当前线程的 ThreadLocalMap 中对应的 Entry
- 如果获取到的 Entry 不为 null，从 Entry 中取出值即为所需访问的本线程对应的实例。如果获取到的 Entry 为 null，则通过 setInitialValue() 方法设置该 ThreadLocal 变量在该线程中对应的具体实例的初始值。
- 这里并不需要考虑 ThreadLocalMap 的线程安全问题。因为每个线程有且只有一个 ThreadLocalMap 对象，并且只有该线程自己可以访问它，其它线程不会访问该 ThreadLocalMap，也即该对象不会在多个线程中共享，也就不存在线程安全的问题。

```
/**
 * Get the entry associated with key.  This method
 * itself handles only the fast path: a direct hit of existing
 * key. It otherwise relays to getEntryAfterMiss.  This is
 * designed to maximize performance for direct hits, in part
 * by making this method readily inlinable.
 *
 * @param  key the thread local object
 * @return the entry associated with key, or null if no such
 */
private Entry getEntry(ThreadLocal<?> key) {
    int i = key.threadLocalHashCode & (table.length - 1);
    Entry e = table[i];
    if (e != null && e.get() == key)
        return e;
    else
        return getEntryAfterMiss(key, i, e);
}
```

---

## 5. 防止内存泄漏
- 对于已经不再被使用且已被回收的 ThreadLocal 对象，它在每个线程内对应的实例由于被线程的 ThreadLocalMap 的 Entry 强引用，无法被回收，可能会造成内存泄漏。
- 针对该问题，ThreadLocalMap 的 set 方法中，通过 replaceStaleEntry 方法将所有键为 null 的 Entry 的值设置为 null，从而使得该值可被回收。另外，会在 rehash 方法中通过 expungeStaleEntry 方法将键和值为 null 的 Entry 设置为 null 从而使得该 Entry 可被回收。通过这种方式，ThreadLocal 可防止内存泄漏。

---

## 6. 案例
- 对于 Java Web 应用而言，Session 保存了很多信息。很多时候需要通过 Session 获取信息，有些时候又需要修改 Session 的信息。一方面，需要保证每个线程有自己单独的 Session 实例。另一方面，由于很多地方都需要操作 Session，存在多方法共享 Session 的需求。如果不使用 ThreadLocal，可以在每个线程内构建一个 Session 实例，并将该实例在多个方法间传递。

```
// 每个需要使用 Session 的地方，都需要显式传递 Session 对象，方法间耦合度较高
public class SessionHandler {
  @Data
  public static class Session {
    private String id;
    private String user;
    private String status;
  }

  public Session createSession() {
    return new Session();
  }

  public String getUser(Session session) {
    return session.getUser();
  }

  public String getStatus(Session session) {
    return session.getStatus();
  }

  public void setStatus(Session session, String status) {
    session.setStatus(status);
  }

  public static void main(String[] args) {
    new Thread(() -> {
      SessionHandler handler = new SessionHandler();
      Session session = handler.createSession();
      handler.getStatus(session);
      handler.getUser(session);
      handler.setStatus(session, "close");
      handler.getStatus(session);
    }).start();
  }
}
```

- 使用 ThreadLocal 重新实现
```
public class SessionHandler {
    public static ThreadLocal<Session> session = ThreadLocal.<Session>withInitial(() -> new Session());
  
    @Data
    public static class Session {
        private String id;
        private String user;
        private String status;
    }
  
    public String getUser() {
        return session.get().getUser();
    }
  
    public String getStatus() {
        return session.get().getStatus();
    }
  
    public void setStatus(String status) {
        session.get().setStatus(status);
    }
  
    public static void main(String[] args) {
        new Thread(() -> {
            SessionHandler handler = new SessionHandler();
            handler.getStatus();
            handler.getUser();
            handler.setStatus("close");
            handler.getStatus();
        }).start();
    }
}
```

---